In [1]:
import numpy as np
import pandas as pd

In [2]:
all_words = np.array(open('../data/scrabble_dict.txt', 'r').read().split('\n'))

In [3]:
puzzle = """
ACNARA
YMTIOC
TAGSCB
JECIAT
GUIOIC
LGRMRE
EDTACN
RUEVAD
"""
n_wds = 6

In [4]:
# Find all words with 4+ letters in grid
lgrid = np.array([list(r) for r in puzzle.split('\n') if r != ''])
grr, grc = lgrid.shape
rr = np.repeat(np.arange(grr)+1, grc)
cc = np.tile(np.arange(grc)+1, grr)
lgrid = np.pad(lgrid, 1, constant_values='#')

found_str = np.expand_dims(lgrid[rr, cc], 1)

avail_grid = np.repeat(-np.ones(tuple([1, *lgrid.shape]), dtype=int), 
                       rr.shape[0], 
                       axis=0)
avail_grid[np.arange(avail_grid.shape[0]), 1:-1, 1:-1] = 0
avail_grid[np.arange(avail_grid.shape[0]), rr, cc] = 1

found_wds = list()
found_wds_map = list()
lnum = 1

while found_str.shape[0] > 0:
    rr = np.repeat(rr, 8) + np.tile([-1, -1, 0, 1, 1, 1, 0, -1], rr.shape[0])
    cc = np.repeat(cc, 8) + np.tile([0, 1, 1, 1, 0, -1, -1, -1], cc.shape[0])
    avail_grid = np.repeat(avail_grid, 8, axis=0)
    filt = avail_grid[np.arange(avail_grid.shape[0]), rr, cc] == 0
    
    rr, cc, avail_grid = [i[filt] for i in(rr, cc, avail_grid)]
    found_str = np.column_stack((np.repeat(found_str, 8, axis=0)[filt], 
                                 lgrid[rr, cc]))
    
    fstr = found_str.reshape(-1).view(f'<U{found_str.shape[1]}')
    lnum += 1
    avail_grid[np.arange(avail_grid.shape[0]), rr, cc] = lnum
    valid_str = np.unique(all_words.view('<U1')
                                   .reshape(all_words.shape[0], -1)[:, :found_str.shape[1]]
                                   .reshape(-1)
                                   .view(f'<U{found_str.shape[1]}'))
    
    vstr_filt = np.isin(fstr, valid_str)
    rr, cc, found_str, fstr, avail_grid = [i[vstr_filt] for i in(rr, cc, found_str, fstr, avail_grid)]

    if lnum >= 4:
        wordfilt = np.isin(fstr, all_words)
        found_wds += fstr[wordfilt].tolist()
        found_wds_map += [avail_grid[wordfilt][:, 1:-1, 1:-1]]

found_wds = np.array(['', *found_wds])
found_wds_map = np.concatenate([np.zeros_like(found_wds_map[0][:1]),
                                *found_wds_map],
                               axis=0)

In [5]:
def do_flood_chk(inarray):
    flood_chk = np.pad(inarray, 
                       ((0, 0), (1, 1), (1, 1)), 
                       'constant', 
                       constant_values=9999)
    while True:
        n_rch = np.sum(flood_chk == -2)
        flood_chk[:, 1:-1, 1:-1] = np.where(((flood_chk[:, 1:-1, 1:-1] <= 0)
                                             & np.any(np.stack([flood_chk[:, 
                                                                1+r_os: flood_chk.shape[1]-1+r_os, 
                                                                1+c_os: flood_chk.shape[2]-1+c_os] == -2
                                                                for r_os in range(-1, 2)
                                                                for c_os in range(-1, 2)
                                                                if r_os != 0 or c_os != 0]),
                                                      axis=0)),
                                            -2,
                                            flood_chk[:, 1:-1, 1:-1])
        if np.sum(flood_chk == -2) == n_rch:
            break
            
    return flood_chk[:, 1:-1, 1:-1] == -1

In [ ]:
# # Get all pairs of non-overlapping words in Pandas DF
# x = np.repeat(np.arange(found_wds.shape[0]), found_wds.shape[0])
# y = np.tile(np.arange(found_wds.shape[0]), found_wds.shape[0])
# pair_ix = np.column_stack((x, y))[y > x]
# for r in range(grr):
#     for c in range(grc):
#         pair_ix = pair_ix[np.any(found_wds_map[:, r, c][pair_ix] == 0, axis=1)]

# # Get all sets of {n_wds-1} non-overlapping words
# pair_ix = pair_ix.merge(pair_ix.rename(columns={'wd1': 'wd2'}), 
#                         on='wd0')\
#                  .merge(pair_ix.rename(columns={'wd0': 'wd1',
#                                                 'wd1': 'wd2'}), 
#                         on=['wd1', 'wd2'])
# pair_ix.shape

In [ ]:
# poss_wds_ix = np.expand_dims(np.arange(found_wds.shape[0]), 1)
# poss_map = np.copy(found_wds_map)

# expix = np.arange(found_wds.shape[0]*poss_wds_ix.shape[0])\
#             [np.tile(np.arange(found_wds.shape[0]), poss_wds_ix.shape[0]) 
#              > np.repeat(poss_wds_ix[:, -1], found_wds.shape[0])]
# possix = np.repeat(poss_wds_ix[:, -1], found_wds.shape[0])[expix]
# addix = np.tile(np.arange(found_wds.shape[0]), poss_wds_ix.shape[0])[expix]
# ovlp_filt = np.all((poss_map[possix] == 0) | (found_wds_map[addix] == 0), axis=(1, 2))

# poss_wds_ix = np.column_stack((poss_wds_ix[possix][ovlp_filt], 
#                                np.arange(found_wds.shape[0])[addix][ovlp_filt]))
# poss_map = poss_map[possix][ovlp_filt]\
#                + np.where(found_wds_map[addix][ovlp_filt] > 0,
#                           found_wds_map[addix][ovlp_filt] + 10,
#                           found_wds_map[addix][ovlp_filt])

In [8]:
poss_wds = np.empty((1, 0), dtype=found_wds.dtype)
poss_map = np.zeros_like(found_wds_map[:1])
poss_wds_done = []
poss_map_done = []

for rr in range(grr):
    for cc in range(grc):
        # Create indices for next word possibilities
        poss_ix = np.repeat(np.arange(poss_map.shape[0]),
                            np.where(poss_map[:, rr, cc] == 0, found_wds.shape[0], 1))
        nw_ix = np.arange(poss_ix.shape[0])\
                - np.maximum.accumulate(np.where(poss_ix != np.append([-1], poss_ix[:-1]), 
                                                 np.arange(poss_ix.shape[0]), 
                                                 0))
        filt = (nw_ix == 0) | (found_wds_map[nw_ix, rr, cc] != 0)
        poss_ix, nw_ix = (i[filt] for i in (poss_ix, nw_ix))
        for r in range(grr):
            for c in range(grc):
                filt = (poss_map[poss_ix, r, c] == 0) | (found_wds_map[nw_ix, r, c] == 0)
                poss_ix, nw_ix = (i[filt] for i in (poss_ix, nw_ix))

        # Generate words list for each possible scenario
        poss_wds = np.column_stack((poss_wds[poss_ix], found_wds[nw_ix]))
        for c in range(poss_wds.shape[1]-2, -1, -1):
            filt = poss_wds[:, c] == ''
            poss_wds[filt, c:-1] = poss_wds[filt, c+1:]
            poss_wds[filt, -1] = ''
        poss_wds = poss_wds[:, :np.sum(np.sum(poss_wds != '', axis=0) > 0)]

        # Count words for each possible scenario
        poss_nwds = np.sum(poss_wds != '', axis=1)

        # Generate grid map for each possible scenario
        poss_map = poss_map[poss_ix]\
                   + np.where(found_wds_map[nw_ix] > 0, 
                              found_wds_map[nw_ix] + (np.expand_dims(poss_nwds, (1, 2)) * 100), 
                              found_wds_map[nw_ix])
        poss_map[nw_ix == 0, rr, cc] = np.where(poss_map[nw_ix == 0, rr, cc] == 0, 
                                                -1, 
                                                poss_map[nw_ix == 0, rr, cc])

        # Filter out scenarios where a -1 is orphaned, cannot be part of pangram
        iso_from = dict()
        iso_from['N'] = do_flood_chk(np.append(-2*np.ones_like(poss_map[:, :1, :]),
                                               poss_map, 
                                               axis=1))[:, 1:, :]
        iso_from['S'] = do_flood_chk(np.append(poss_map,
                                               -2*np.ones_like(poss_map[:, :1, :]),
                                               axis=1))[:, :-1, :]
        iso_from['W'] = do_flood_chk(np.append(-2*np.ones_like(poss_map[:, :, :1]),
                                               poss_map, 
                                               axis=2))[:, :, 1:]
        iso_from['E'] = do_flood_chk(np.append(poss_map,
                                               -2*np.ones_like(poss_map[:, :, :1]),
                                               axis=2))[:, :, :-1]
        filt = ~np.any((iso_from['N'] | iso_from['S']) & (iso_from['W'] | iso_from['E']), 
                       axis=(1, 2))
        poss_wds = poss_wds[filt]
        poss_nwds = poss_nwds[filt]
        poss_map = poss_map[filt]        

        # Remove scenarios where required number of words reached
        filt = poss_nwds == n_wds
        poss_wds_done += [poss_wds[filt]]
        poss_map_done += [poss_map[filt]]
        poss_wds = poss_wds[~filt]
        poss_map = poss_map[~filt]

        print(rr, cc, poss_wds.shape[0])

0 0 30
0 1 79
0 2 1187
0 3 3389
0 4 9926
0 5 13359
1 0 24075
1 1 36346
1 2 67047
1 3 75200
1 4 99998
1 5 94522
2 0 184157
2 1 271258
2 2 299586
2 3 1090681
2 4 1405278


MemoryError: Unable to allocate 7.46 GiB for an array with shape (1001737288,) and data type int64

In [21]:
poss_map[1000000]

array([[102, 101,  -1,  -1,  -1,  -1],
       [ -1, 103,  -1,  -1,  -1,  -1],
       [ -1, 104,  -1, 203,  -1,   0],
       [  0,   0, 204,   0, 202,   0],
       [  0,   0,   0, 205,   0, 201],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0]])

In [20]:
poss_wds[1000000]

array(['CAMA', 'CASCO', '', '', '', ''], dtype='<U10')

In [15]:
poss_wds_done[-1][0]

array(['ACTA', 'NISI', 'ACRO', 'GAMY', 'TEGU', 'CACTI'], dtype='<U10')

In [16]:
poss_map_done[-1][0]

array([[101, 102, 201, 104, 303, 301],
       [404, 403, 103, 202, 304, 302],
       [501, 402, 401, 203, 601,   0],
       [  0, 502,   0, 204, 602, 604],
       [503, 504,   0,   0, 605, 603],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0]])

In [ ]:
# 0 0 30
# 0 1 83
# 0 2 1541
# 0 3 4999
# 0 4 17595
# 0 5 27904
# 1 0 53133
# 1 1 88391